In [1]:
!pip install langchain openai
!pip install langchain-experimental langchain-openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the

## LLM

In [4]:
from getpass import getpass

OPENAI_API_KEY = getpass()

import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

··········


In [2]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import OpenAI

db = SQLDatabase.from_uri("sqlite:///employee_dept.db")

In [3]:
db.get_table_info()

'\nCREATE TABLE departments (\n\tdepartment_id INTEGER, \n\tdepartment_name TEXT NOT NULL, \n\tmanager_id INTEGER, \n\tlocation_id INTEGER, \n\tPRIMARY KEY (department_id)\n)\n\n/*\n3 rows from departments table:\ndepartment_id\tdepartment_name\tmanager_id\tlocation_id\n10\tAdministration\t200\t1700\n20\tMarketing\t201\t1800\n30\tPurchasing\t114\t1700\n*/\n\n\nCREATE TABLE employees (\n\temployee_id INTEGER, \n\tfirst_name TEXT, \n\tlast_name TEXT NOT NULL, \n\temail TEXT NOT NULL, \n\tphone_number INTEGER, \n\thire_date TEXT NOT NULL, \n\tjob_id TEXT NOT NULL, \n\tsalary REAL, \n\tcommission_pct REAL, \n\tmanager_id INTEGER, \n\tdepartment_id INTEGER, \n\tPRIMARY KEY (employee_id), \n\tFOREIGN KEY(department_id) REFERENCES departments (department_id), \n\tUNIQUE (email)\n)\n\n/*\n3 rows from employees table:\nemployee_id\tfirst_name\tlast_name\temail\tphone_number\thire_date\tjob_id\tsalary\tcommission_pct\tmanager_id\tdepartment_id\n100\tSteven\tKing\tSKING\t515.123.4567\t17-JUN-03\t

#Case 1: Text-to-SQL query

In [7]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "Find top salaried employee in each department"})
print(response)

SELECT departments.department_name, employees.first_name, employees.last_name, MAX(employees.salary) 
FROM departments 
JOIN employees ON departments.department_id = employees.department_id 
GROUP BY departments.department_name 
ORDER BY MAX(employees.salary) DESC 
LIMIT 5;


#Case 2: Text-to-SQL query and execution

In [5]:
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

db_chain.run("How many employees are there?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM employees
SQLResult: [(50,)]
Answer:50
> Finished chain.


'50'

#Case 3: SQL agents

In [8]:
from langchain.agents import create_sql_agent

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain_community.agent_toolkits import SQLDatabaseToolkit


agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
agent_executor.run(
    "Which department has highest expenditure in term of total salary paid?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: departments, employees
Thought: I should query the schema of the departments and employees tables to see what columns I can use.
Action: sql_db_schema
Action Input: departments, employees
Observation: 
CREATE TABLE departments (
	department_id INTEGER, 
	department_name TEXT NOT NULL, 
	manager_id INTEGER, 
	location_id INTEGER, 
	PRIMARY KEY (department_id)
)

/*
3 rows from departments table:
department_id	department_name	manager_id	location_id
10	Administration	200	1700
20	Marketing	201	1800
30	Purchasing	114	1700
*/


CREATE TABLE employees (
	employee_id INTEGER, 
	first_name TEXT, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	phone_number INTEGER, 
	hire_date TEXT NOT NULL, 
	job_id TEXT NOT NULL, 
	salary REAL, 
	commission_pct REAL, 
	manager_id INTEGER, 
	department_id INTEGER, 
	PRIMARY KEY (employee_id), 
	FOREIGN KEY(department_id) REFERENCES departments (department_id), 
	U

'Shipping'